In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# lib
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

# preprocessing
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

# training
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV

# metric
from sklearn.metrics import log_loss
from sklearn.metrics import roc_auc_score

# model
import lightgbm as lgb
import xgboost as xgb
from sklearn.ensemble import RandomForestClassifier
import catboost
from catboost import CatBoostClassifier

# model save & load
import joblib 
from tensorflow.keras.models import load_model


import warnings
warnings.filterwarnings('ignore')

In [ ]:
# reduce memory
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))

    return df

In [ ]:
og_test = pd.read_csv('../input/traintest/fp_test_dataset.csv')
og_train = pd.read_csv('../input/traintest/fp_train_dataset.csv')

In [ ]:
og_test = reduce_mem_usage(og_test)
og_train = reduce_mem_usage(og_train)

원본 데이터셋을 합쳐서 특정 컬럼을 드랍하고 레이블 인코딩을 하는 등 전처리 과정이 필요했습니다.  
때문에 전처리 전의 데이터셋은 'og_'라는 단어를 앞에 붙여서 표기했습니다.

In [ ]:
print(og_train.shape)
og_train

In [ ]:
print(og_test.shape)
og_test

In [ ]:
og_train.columns

In [ ]:
dataset = pd.concat([og_train, og_test])
dataset.reset_index(drop = True, inplace = True)
dataset

전처리를 위해 하나의 데이터셋으로 만들어야 했기 때문에 train/test 데이터셋을 세로 방향으로 합칩니다.  

In [ ]:
dataset.drop(['Unnamed: 0', 'server_time_kst', 'device_type', 'content_name', 'content_keyword'], axis = 1, inplace = True)
dataset

모든 값이 통일 된 값을 갖고 있는 device_type와 학습에 불필요하다고 판단되거나 EDA과정에서 학습에서 제외하기로 정한 content_name, content_keyword, server_time_kst 컬럼을 드랍합니다.

In [ ]:
dataset.columns

위에 출력된 컬럼들이 모델에 들어갈 컬럼들입니다.

In [ ]:
# label encoding
categorical_features = ['imp_id', 'content_id', 'c_user_gender', 'content_flag_used', 'c_content_category_id_1',
                       'c_content_category_id_2', 'c_content_category_id_3', 'hour']

for feat in categorical_features:
    lbe = LabelEncoder()
    dataset[feat] = lbe.fit_transform(dataset[feat])  

dataset.head()

categorical feature를 label encoding합니다. catboost를 제외한 나머지 앙상블 모델의 경우 numeric한 feature만 사용이 가능하기 때문에  
one-hot encoding을 하거나 label encoding을 해주어야합니다.  

In [ ]:
dataset.info()

위 과정을 거쳐 Ensemble Model에 적용이 가능한 데이터셋이 완성되게 됩니다.

In [ ]:
# 평균 ctr
label_distribution = dataset.groupby('label').size()
print(label_distribution)
avg_ctr = float(label_distribution[1] / label_distribution.sum())
avg_ctr

모델을 돌릴 때 information gain도 함께 측정하기 위해 평균 ctr을 구해봅니다.  
0.03은 3%입니다.

In [ ]:
train = dataset[:575192]
test = dataset[575192:]
test.reset_index(drop = True, inplace = True)

del og_train
del og_test

원본 train/test dataset과 동일하게 8:2로 다시 자릅니다.  
concat할 때 세로 방향으로 붙였기 때문에 row 수만 동일하게 자르면 원본 데이터셋과 동일한 구성이 됩니다.

In [ ]:
X_train = train.drop('label', axis = 1)
y_train = train['label']
X_test = test.drop('label', axis = 1)
y_test = test['label']

del train
del test

print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

In [ ]:
# save & download
def save_download(model, name):
    joblib.dump(model, './' + name)

# load 
# loaded_model = joblib.load('./knn_model.pkl')
# score = loaded_model.score(X,y)
# print('정확도: {score:.3f}'.format(score=score))

In [ ]:
# Model Define

xgbm = xgb.XGBClassifier(learning_rate=0.1,             
                         n_estimators = 1000,           
                         max_depth = 9,                 
                         min_child_weight = 1,          
                         gamma = 0.3,                    
                         n_jobs = -1,                   
                         eval_metric = 'logloss',       
                         objective = 'binary:logistic', 
                         use_label_encoder = False)     

lgbm = lgb.LGBMClassifier(learning_rate = 0.09,
                          n_estimators = 2000,
                          max_depth = 8,
                          num_leaves = 50,
                          min_child_weight = 1,
                          subsample = 0.5,
                          colsample_bytree = 0.5,
                          n_jobs = -1,
                          objective = 'binary',
                          metric = 'binary_logloss',)


cbm = CatBoostClassifier(learning_rate = 0.05,
                         n_estimators = 1600,
                         max_depth = 9)

rfm = RandomForestClassifier(n_estimators = 2000,
                             min_samples_leaf = 4,
                             min_samples_split = 8,)

모델을 정의하고 각 모델의 파라미터를 튜닝합니다.
각 모델을 default 파라미터로 설정한 후에 첫 학습을 돌리고 loss를 확인합니다.  
이 후 각 모델별로 gridsearchCV에 하이퍼 파라미터 범위를 명시해서 튜닝을 시작합니다.  
catboost의 경우 파라미터 최적화가 잘 되어 있는 모델이었기 때문에 많은 하이퍼 파라미터를 사용하는 것 보다 핵심 파라미터만 튜닝하는 것이 성능이 더 좋았습니다.  
randomforest는 트리 수를 최대한 늘리고 분기 조건과 leaf node의 최소 데이터 개수를 조정해가며 overfitting을 제어했습니다.


In [ ]:
# K-Fold for CV
kf = KFold(random_state = 42,
           n_splits = 2,
           shuffle = True)


# Model's hyperparameters for GridSearchCV
# rf_params ={}

# lgb_params ={}

xgb_params ={}

# cb_params = {} 

In [ ]:
# GridSearchCV
gcv = GridSearchCV(estimator = lgbm,
                   param_grid = lgb_params,
                   cv = kf,
                   n_jobs = -1,
                   verbose = 3,
                   scoring = 'neg_log_loss') 

gcv.fit(X_train, y_train)

print(gcv.best_params_)
print(gcv.best_score_)

neg_log_loss가 쓰인 이유는 gridsearchCV에서는 스코어가 높을수록 좋다라고 판단하기 때문입니다. 하지만 log_loss는 낮을수록 좋은 metric이기 때문에 -1를 곱해서 높을수록 좋다고 판단하도록 만든 것입니다.

In [ ]:
# LightGBM
lgbm.fit(X_train, y_train)


prior = log_loss(y_train, [avg_ctr]*len(y_train))

# pred & loss
pred_proba = lgbm.predict_proba(X_test)
pred = lgbm.predict(X_test)
logloss = log_loss(y_test, pred_proba)
AUC = roc_auc_score(y_test, pred)

# relative information gain
rig = (prior - logloss) / prior


print(f'test AUC : {AUC}')
print(f'test log loss : {logloss}')
print(f'rig : {rig}')
print('\n')

# feature importance plot
print('LightGBM : feature importance')
fig, ax = plt.subplots(figsize = (14, 7))
lgb.plot_importance(lgbm, ax=ax)

In [ ]:
save_download(lgbm, 'lgbm_0.0927')

In [ ]:
# XGBoost
xgbm.fit(X_train, y_train)


prior = log_loss(y_train, [avg_ctr]*len(y_train))

# pred & loss
pred_proba = xgbm.predict_proba(X_test)
pred = xgbm.predict(X_test)
logloss = log_loss(y_test, pred_proba)
AUC = roc_auc_score(y_test, pred)

# relative information gain
rig = (prior - logloss) / prior


print(f'test AUC : {AUC}')
print(f'test log loss : {logloss}')
print(f'rig : {rig}')
print('\n')


# feature importance plot
print('XGBoost : feature importance')
fig, ax = plt.subplots(figsize = (14, 7))
xgb.plot_importance(xgbm, ax=ax)

In [ ]:
save_download(xgbm, 'xgbm_auc_0.5577')

In [ ]:
# CatBoost
cbm.fit(X_train, y_train, verbose = False)


prior = log_loss(y_train, [avg_ctr]*len(y_train))

# pred & loss
pred_proba = cbm.predict_proba(X_test)
pred = cbm.predict(X_test)
logloss = log_loss(y_test, pred_proba)
AUC = roc_auc_score(y_test, pred)

# relative information gain
rig = (prior - logloss) / prior


print(f'test AUC : {AUC}')
print(f'test log loss : {logloss}')
print(f'rig : {rig}')
print('\n')


# print('CatBoost : feature importance')

# feature importance plot
def plot_feature_importance(importance,names,model_type):
    
    feature_importance = np.array(importance)
    feature_names = np.array(names)
    
    data={'feature_names':feature_names,'feature_importance':feature_importance}
    fi_df = pd.DataFrame(data)
    
    fi_df.sort_values(by=['feature_importance'], ascending=False,inplace=True)

    plt.figure(figsize=(10,8))

    sns.barplot(x=fi_df['feature_importance'], y=fi_df['feature_names'])

    plt.title(model_type + ' Feature Importance')
    plt.xlabel('Feature Importance')
    plt.ylabel('Feature Names')
    
# plot_feature_importance(catboost.get_feature_importance(),X_test.columns,'CATBOOST')

In [ ]:
save_download(cbm, 'cbm_0.0918')

In [ ]:
# RandomForestClassifier
rfm.fit(X_train, y_train)


prior = log_loss(y_train, [avg_ctr]*len(y_train))

# pred & loss
pred_proba = rfm.predict_proba(X_test)
pred = rfm.predict(X_test)
logloss = log_loss(y_test, pred_proba)
AUC = roc_auc_score(y_test, pred)

# relative information gain
rig = (prior - logloss) / prior


print(f'test AUC : {AUC}')
print(f'test log loss : {logloss}')
print(f'rig : {rig}')
print('\n')
# print('RandomForestClassifier : feature importance')

# feature importance plot
# fig, ax = plt.subplots(figsize = (14, 7))
# lgb.plot_importance(lgbm, ax=ax)

In [ ]:
save_download(rfm, 'rfm_0.0942')

In [ ]:
# 임의 유저로 테스트해보자.

# 모델에 사용한 dataset에서 유저 정보를 드랍한다.
seonwoo_dataset = dataset.drop(['c_user_gender', 'c_user_age',
       'user_following_count', 'user_pay_count', 'user_parcel_post_count',
       'user_transfer_count', 'user_chat_count'], axis = 1)

# 임의 유저의 정보를 추가한다.
seonwoo_dataset['c_user_gender'] = 1
seonwoo_dataset['c_user_age'] = 34
seonwoo_dataset['user_following_count'] = 0
seonwoo_dataset['user_pay_count'] = 0
seonwoo_dataset['user_parcel_post_count'] = 0
seonwoo_dataset['user_transfer_count'] = 0
seonwoo_dataset['user_chat_count'] = 0

seonwoo_dataset_label = seonwoo_dataset['label']
seonwoo_dataset.drop('label', axis = 1, inplace = True)
seonwoo_dataset

실제 추천을 하기 위해 임의 유저의 데이터를 입력하는 부분입니다.  
학습에 사용된 데이터셋에서 유저 정보와 관련된 컬럼을 모두 드랍하고 대신 임의의 유저 정보를 입력해서 컬럼을 다시 채웁니다.  
나머지 컬럼의 값들은 모두 동일하기 때문에 unseen data(유저 정보)를 넣어주고 모델이 추론하는 것과 같습니다.    
현재 cell은 신규 유저를 기준으로 예측을 하기 위한 유저 정보를 입력한 것입니다.

In [ ]:
compression_opts = dict(method='zip',
                        archive_name='가상인물데이터2.csv')  
seonwoo_dataset.to_csv('가상인물데이터2.zip', index=False,
          compression=compression_opts)

In [ ]:
seonwoo_dataset.isnull().sum()

In [ ]:
# 임의 유저에 대해 prediction을 해보자.
probs = xgbm.predict_proba(seonwoo_dataset)

# 계산된 확률을 dataframe에 추가하자.
seonwoo_dataset['probs'] = probs[:, 1]

seonwoo_dataset

AUC상 가장 성능이 좋았던 학습된 xgboost 모델을 이용해서 위에서 만든 임의 유저 데이터셋을 예측합니다.  
예측된 값(확률 값)을 컬럼에 추가합니다.  
확률이 높은 순으로 top-k개를 잘라내야 하기 때문입니다.

In [ ]:
compression_opts = dict(method='zip',
                        archive_name='가상인물데이터2 예측 결과.csv')  
seonwoo_dataset.to_csv('가상인물데이터2 예측 결과.zip', index=False,
          compression=compression_opts)

In [ ]:
seonwoo_loss = log_loss(seonwoo_dataset_label, probs)
seonwoo_loss

임의 유저에 대해 진행한 prediction의 logloss를 확인합니다.

In [ ]:
# 임의 유저가 보게 될 확률이 높은 광고주 top-3

recommended = seonwoo_dataset.sort_values(by = ['probs'], ascending = False).drop_duplicates(['advertiser_id'], keep = 'first').head(3)
recommended

우리 데이터셋의 특성상 top-n개를 뽑으면 같은 동일한 광고주가 추천될 가능성을 제외하기 위해 advertiser_id 중복을 제거하고 내림차순 top-3를 뽑습니다.

In [ ]:
compression_opts = dict(method='zip',
                        archive_name='가상인물데이터2 top-3 노편집.csv')  
recommended.to_csv('가상인물데이터2 top-3 노편집.zip', index=False,
          compression=compression_opts)

In [ ]:
rec_adv = pd.DataFrame(recommended['advertiser_id'])
rec_adv

top-3까지 잘린 데이터프레임에서 advertiser_id만 뽑아내면 임의의 유저가 볼 확률이 높은 광고주 top-3 명단이 만들어집니다.

In [ ]:
# rec_adv.to_csv(index = False)

compression_opts = dict(method='zip',
                        archive_name='out2.csv')  
rec_adv.to_csv('xgb_output2.zip', index=False,
          compression=compression_opts)

In [ ]:
# plot

auc_df = pd.DataFrame({"RandomForest": [0.5061],
                       "LightGBM" : [0.5420],
                       'XGBoost' : [0.5577],
                       'CatBoost' : [0.5167]})

auc_df.rename(index = {0:'AUC'}, inplace = True)
auc_df
# sns.lineplot(data = auc_df)

각 모델의 AUC score를 기준으로 plot을 만들기 위해 데이터프레임을 만듭니다.  

In [ ]:
# plot
sns.set_style('whitegrid')
sns.set(rc = {'figure.figsize' : (15, 8)},
        font_scale = 2)
sns.lineplot(data = auc_df.T.reset_index().rename(columns = {'index' : 'model'}), 
             x = 'model', 
             y = 'AUC',
             color = 'r',
             linewidth = 3.5)
plt.title('AUC Score by Gradient Boosting Model')
plt.grid(True)

plt.savefig('auc_plot.png')

